This script generates hillslope mesh with a varying organic layer thickness at the top.

In [4]:
# Importing all the package needed to run teh watershed workflow
import os
import numpy as np
from matplotlib import pyplot as plt
import shapely
import logging
import copy
import sys
import exodus3
import watershed_workflow 
import watershed_workflow.source_list
import watershed_workflow.ui
import watershed_workflow.utils
import watershed_workflow.plot
import watershed_workflow.mesh
import watershed_workflow.regions
import watershed_workflow.densification
import watershed_workflow.condition
watershed_workflow.ui.setup_logging(1,None)
crs=None

In [5]:
###########################################
# Define the grid size and range
min_x, max_x = 0.0, 100.0
min_y, max_y = 0.0, 1.0
Range= shapely.geometry.Polygon(np.array([[min_x, min_y],[max_x, min_y],[max_x, max_y],[min_x, max_y]]))
dx, dy = 2, 1
nx = int((max_x)/dx)
ny = int((max_y)/dy)

# Create the grid points
x = np.arange(min_x, max_x + dx, dx)
y = np.arange(min_y, max_y + dy, dy)

In [6]:
# Create the cells and store the nodes and connectivity
nodes = []
connectivity = []
node_id = 0
node_dict = {}
for i in range(len(x)-1):
    for j in range(len(y)-1):
        # Define the vertices of the cell
        cell_points = [(x[i], y[j]), (x[i+1], y[j]), (x[i+1], y[j+1]), (x[i], y[j+1])]
        cell_polygon = shapely.geometry.Polygon(cell_points)
        
        # Check if the cell intersects the range
        if cell_polygon.intersects(Range):
            cell_intersection = cell_polygon.intersection(Range)
            if isinstance(cell_intersection, shapely.geometry.Polygon):
                # Store the nodes with IDs
                cell_node_ids = []
                for point in cell_points:
                    if point not in node_dict:
                        node_dict[point] = node_id
                        nodes.append([point[0], point[1], 0])
                        node_id += 1
                    cell_node_ids.append(node_dict[point])
                connectivity.append(cell_node_ids)

# Convert lists to numpy arrays
nodes = np.array(nodes)
connectivity = connectivity

In [7]:
# Make 2D mesh
def dem_(x, y):
    if x <= 25: 
        z = 100 - (1.0*x)/25
    elif x >= 75:
        z = 96.0 - (1.0*np.abs(75-x))/25
    else:
        z = (99.0 - 96.0)/(25-75) * (x-25) + 99.0

    if (y == 2) | (y == 10):
        z -=0.1
    elif (y == 4) | (y == 8):
        z -=0.2
    elif (y == 6):
        z -= 0.3
    return z

In [8]:
x_comp = nodes[:, 0]; y_comp = nodes[:, 1]
nodes[:, 2] = np.array([dem_(x_comp[i], y_comp[i]) for i in range(len(x_comp))])
m2 = watershed_workflow.mesh.Mesh2D(nodes, connectivity)

###########################################
# Save dem values
coordinate = m2.centroids[:, :2]
ats_dem = np.array([dem_(coordinate[i, 0],  coordinate[i, 1]) for i in range(len(coordinate))])
ats_dem = ats_dem.reshape(nx, ny).T.reshape(-1)
ats_dem = ats_dem.reshape(1, ny, nx) 
ats_dem = np.array(ats_dem, dtype='d')
# np.save("dem_values.npy", ats_dem)

###########################################
# Set surface regions
## Surface : Plateau - 11 / Slope - 12 / Baxio - 13
veg_labels_dict = dict(zip([11, 12, 13], ['Plateau', 'Slope', 'Baxio']))

veg_colors = []
for i in range(len(m2.centroids)):
    if m2.centroids[i, 0] <= 25:
        veg_colors.append(11)
    elif m2.centroids[i, 0] >= 75:
        veg_colors.append(13)
    else:
        veg_colors.append(12)
        
veg_colors = np.array(veg_colors)

# This source code from the .regions.add_nlcd_labeled_sets 
inds = np.unique(veg_colors)

if veg_labels_dict is None:
    veg_labels_dict = dict((i, str(i)) for i in inds)

for ind in inds:
    ent_ids = list(np.where(veg_colors == ind)[0])
    ls = watershed_workflow.mesh.LabeledSet(veg_labels_dict[ind], int(ind), 'CELL', ent_ids)
    m2.labeled_sets.append(ls)

In [15]:
# Creating a 3D mesh
## When making the mesh, please check all mesh slope should not be zero.
def layer2(x, y):
    if x <= 25:
        return 1.2 - 0.2 * (x/25)
    elif x >= 75: 
        return 0.4 - 0.2 * (x-75)/25
    else: 
        return 0.4 + 0.6*(75-x)/(75-25)

def layer3(x, y):
    return 0.3 + 0.7*(100-x)/(100)

dzs= sum([[0.05]*10, [0.1]*5, [0.2]*10, [layer2]*3, [layer3]*2], [])
nz = len(dzs)

# layer extrusion
# -- data structures needed for extrusion
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []

for i, dz in enumerate(dzs):
    if i <= 24:
        layer_types.append('constant')
        layer_data.append(dz)
        layer_mat_ids.append(1001)
    else:
        layer_types.append('function')
        layer_data.append(dz)
        layer_mat_ids.append(1001)
        
    layer_ncells.append(1)
    
m3 = watershed_workflow.mesh.Mesh3D.extruded_Mesh2D(m2, layer_types, layer_data,layer_ncells, layer_mat_ids)

###########################################
# Save dz values
dz_values_2 = [layer2(m2.centroids[i, 0], m2.centroids[i, 1]) for i in range(len(m2.centroids))]
dz_values_2 = np.round(dz_values_2, 2)
dz_values_2 = list(dz_values_2.reshape(nx, ny).T.reshape(-1))

dz_values_3 = [layer3(m2.centroids[i, 0], m2.centroids[i, 1]) for i in range(len(m2.centroids))]
dz_values_3 = np.round(dz_values_3, 2)
dz_values_3 = list(dz_values_3.reshape(nx, ny).T.reshape(-1))

ats_dz = np.array(sum([dz_values_3 * 2,  dz_values_2*3, [0.2]*ny*nx*10, [0.1]*ny*nx*5, [0.05]*ny*nx*10], []))
ats_dz = ats_dz.reshape(nz, ny, nx)
# np.save("dz_values.npy", ats_dz)

###########################################
# Set subsurface regions
mat_c = []
for i in range(len(m2.centroids)):
    if m2.centroids[i, 0] <= 25:
        mat_c.append(101)
    elif m2.centroids[i, 0] >= 75:
        mat_c.append(103)
    else:
        mat_c.append(102)

mat_ids = []
for i in range(30): # dz layers 
    mat_ids.append(mat_c)
mat_ids = np.array(mat_ids)

#### Subsurface : Plateau - 101 / Slope - 102 / Baxio - 103
mesh2D = m2
layer_types = layer_types
layer_data = layer_data
ncells_per_layer = layer_ncells
np_mat_ids = mat_ids

material_ids = np.zeros((m3.num_cells, ), 'i')

ncells_tall = sum(ncells_per_layer)
ncells_total = ncells_tall * mesh2D.num_cells
cells = [list() for c in range(ncells_total)]

eh = watershed_workflow.mesh._ExtrusionHelper(ncells_tall, mesh2D.num_cells)

for col in range(mesh2D.num_cells):
    z_cell = 0
    for ilay in range(len(ncells_per_layer)):
        ncells = ncells_per_layer[ilay]
        for i in range(z_cell, z_cell + ncells):
            material_ids[eh.col_to_id(col, i)] = np_mat_ids[ilay, col]
        z_cell = z_cell + ncells

m3.material_ids = material_ids
m3.material_ids_list[:] = list(np.unique(material_ids))

In [ ]:
# Save mesh 
filename = '../data/gradient_hillslope2.exo'
if os.path.exists(filename):
    os.remove(filename)
m3.write_exodus(filename)